In [2]:
from astropy.modeling import models
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
from astropy import units as u
from astropy import constants as c

## Problem 13.5 ###

### Given: ###
Star HD 209458 has the following properties:
* M_s = 1.1 M_\odot
* L_s = 1.6 L_\odot
* T_s = 6000 K

It is orbited by a planet with
* M_p = 0.69 M_{jup}
* R_p = 1.35 R_{jup}
* d = 0.045 AU


### Find: ####

Would hydrogen evaportate from the star?


### Approach: ###

* Solve for escape temperature and temperature of the planet.  If $T_{planet} > T_{esc}$, then gas will evaporate.
* Need to estimate the albedo - will use the albedo of gas giants, $a \approx 0.5$.

In [3]:
#star properties
M_s = 1.1 *c.M_sun
L_s = 1.6 *c.L_sun
T_s = 6000 *u.K

# planet properties
M_p = 0.69 * c.M_jup
R_p = 1.35 * c.R_jup
d = 0.045*c.au
a = 0.5


### Temperature of Planet ###

Use Eqn 13.15

$$ T_p = T_s (1-a)^{1/4} \left(\frac{R_s}{2D}\right)^{1/2} $$

Need to solve for radius of Sun using BB equation:
$$ L = 4 \pi R_s^2 \sigma T^4 $$

So $$R_s = \sqrt{\frac{L}{4 \pi \sigma T^4}}$$

In [4]:
R_s = np.sqrt(L_s/(4*np.pi*c.sigma_sb*T_s**4))
print(R_s)

814390950.0410734 m


In [5]:
# From textbook Eqn 13.15
T_p = (1-a)**(.25)*T_s*np.sqrt(R_s/2/d)
print(T_p)

1240.869929267471 K


### Escape Temperature ###

$$T_{esc} = \frac{1}{54} \frac{G M m}{k R} $$

* M is the mass of the planet
* m is the mass of the gas atom in question, molecular hydrogen or $H_2$ ($2 \times m_{proton}$)


In [6]:
m_H2 = 2*c.m_p
T_esc = 1./54*c.G*M_p* m_H2/(c.k_B*R_p)
print('T_esc = %.2f %s'%(T_esc.to('K').value,T_esc.to('K').unit))

T_esc = 4063.86 K


### Conclusion ###

The planet is hot ($T = 1240$~K), but it is not hot enough to evaporate $H_2$, which has an escape temperature of $T = 4064$~K.

## Problem 13.6  - Detecting Light from Planets##


### Given:###

* set of wavelengths
* temp and luminosity of the star
* temp and radius of the planet

### Find: ###

* luminosity output of star vs planet as a function of wavelength

### Aproach: ###

* calculate the luminosity of the star and planet.
* Use astropy's blackbody1D function to calculate the flux output at each wavelength.  
* calculate the ratio of $L_{star}/L_{planet}$ as a function of wavelength.


In [7]:
wavelengths = np.array([450., 700.,2200.,10000.]) # wavelengths is nm

In [8]:
Tstar = 6000 * u.K
Lstar = 1.61*c.L_sun
bbstar = models.BlackBody1D(Tstar, Lstar)

In [9]:
Tplanet = 1500. * u.K
Rplanet = 1.35 * c.R_jup
Lplanet = c.sigma_sb * Tplanet**4 * 4 * np.pi * Rplanet**2
a = .5 #albedo of the planet
bbplanet = models.BlackBody1D(Tplanet, Lplanet)
# reflected light
d = 0.045*c.au


In [22]:
for w in wavelengths:
    print('wavelength = ',w)
    s = bbstar(w*u.nm)
    p = bbplanet(w*u.nm) + a*s*(Rplanet/(4*d))**2
    #print(s)
    #print(p)
    print('\t star        = ',s)
    print('\t planet      = ',p)
    print('\t star/planet = ',s.value/p.value)
    print('\t planet/star = ',p.value/s.value)

wavelength =  450.0
	 star        =  559811050493.7655 W / Hz
	 planet      =  3596708.0360157457 W / Hz
	 star/planet =  155645.39709314198
	 planet/star =  6.424860732640721e-06
wavelength =  700.0
	 star        =  1026030373401.1783 W / Hz
	 planet      =  7067317.626312279 W / Hz
	 star/planet =  145179.6038685416
	 planet/star =  6.88801989641388e-06
wavelength =  2200.0
	 star        =  497936507623.811 W / Hz
	 planet      =  180810438.70756003 W / Hz
	 star/planet =  2753.9146035100644
	 planet/star =  0.00036311946591423977
wavelength =  10000.0
	 star        =  38626446594.90183 W / Hz
	 planet      =  91017496.76195738 W / Hz
	 star/planet =  424.3848487277508
	 planet/star =  0.0023563517948340207


Conclusion:  The ratio of starlight to planet light is lowest in infrared wavelengths, so that is where you should try to image the planet.  In fact, astronomers who are trying to take direct images of planets usually work at 10 microns.

## Problem 13.8  - Eris ##


### Part (a) ###

**Given:**

* model from Sect. 13.2



* radius $R_p$
* distance from Sun, $D$


**Find:**

* Luminosity of the planet, $L_p$

**Approach:**

* The amount reflected is the albedo times the amount incident:

$$L_{reflected} = a L_{incident} $$

* The amount incident is the flux of the star times the cross section of the planet:

$$L_{incident} = flux \times \pi R_p^2 = \frac{L_\odot}{4 \pi D^2} \pi R_p^2 $$

$$L_{incident} =  \frac{L_\odot R_p^2}{4 D^2} $$

* And the reflected light is thus:


$$L_{reflected} =  \frac{a L_\odot R_p^2}{4 D^2} $$


### Part (b) ###

**Given:**
* $D = 97$ AU
* $L_p = 5.8 \times 10^{11}~J~s^{-1}$

**Find:**
* $R_p$, with upper and lower limits

**Approach:**

* Solve above equation for $R_p$:


$$ R_p = 2 D \sqrt{\frac{L_p }{ a L_\odot}}$$

* need to estimate a realistic range for albedo.  Using upper and lower estimate for albedo will yield a range in $R_p$

* For comparison, the albedo of Pluto is $0.44-0.61$ according to https://en.wikipedia.org/wiki/Bond_albedo

* Mercury has the lowest albedo $a = 0.142$, and Venus has the highest albedo $a = 0.689$.


In [44]:
D = 97.*u.AU
Lp = 5.8e11*u.J/u.s
Tsun = 5800.*u.K
# Estimate albedo
amin = 0.142
amax = 1. # upper bound

In [45]:
a = amin
Rp = 2*D.to('m')*np.sqrt(Lp/a/c.L_sun)
print('Rp min = %.2e %s'%(Rp.value,Rp.to('m').unit))

Rp min = 3.00e+06 m


In [46]:
a = amax
Rp = 2*D.to('m')*np.sqrt(Lp/a/c.L_sun)
print('Rp min = %.2e %s'%(Rp.value,Rp.to('m').unit))

Rp min = 1.13e+06 m


### Part (c) ###

**Given: **

* peak wavelength $\lambda_{peak} = 116 ~\mu m$

**Find: **

* effective temperature

**Approach:**

* Assume Eris is a blackbody, so we can apply Wien's Law

$$ \lambda_{peak} T = 2.9 \times 10^{-3}~m~K $$

In [47]:
lambda_max = 116*u.micron
Tp = 2.9e-3*u.m*u.K/lambda_max.to('m')
print('Effective temp = %.0f %s'%(T.value,T.unit))

Effective temp = 25 K


### Part (d) ###

**Find:**

* revised estimate for $R_p$


**Approach:**

* Use Eqn 13.15, which gives the planet temperature 

$$ T_p = T_\odot (1-a)^{1/4} \left(\frac{R_\odot}{2D}\right)^{1/2} $$

* Solve for albedo, and use temp from part (c) to estimate the albedo.

$$ a = 1 - \left(\frac{T_p}{T_\odot} \right)^4 \left(\frac{2 D}{R_\odot} \right) ^2 $$

* Use revised albedo to estimate the planet's radius, as we did in part (b).

In [50]:
a = 1 - (Tp/Tsun)**4 * (2*D/c.R_sun)**2
print('albedo = %.3f'%(a))

albedo = 0.399


In [51]:
# revised estimate of planet's radius
Rp = 2*D.to('m')*np.sqrt(Lp/a/c.L_sun)
print('Rp min = %.2e %s'%(Rp.value,Rp.to('m').unit))

Rp min = 1.79e+06 m
